In [ ]:
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Callable
from utils import llm_call
import re 
import os 
from openai import OpenAI
from groq import Groq

In [ ]:
user_input1 = '저녁메뉴 1개만 추천해줘'

user_input2 = "ChatGPT와 일반 챗봇의 차이점을 간단히 설명해줘."

user_input3 = "OverFitting의 개념을 알려주고 해결 방법에는 어떤 것들이 있는지 구체적인 예시를 들어서 상세히 알려줘"


def route(user_input : str) -> str:
    select_prompt = f'''

    user_input : {user_input}


    당신은 AI 라우팅 시스템입니다.
    사용자의 요청을 보고 가장 적절한 AI 모델을 선택하세요.

    선택 가능한 모델:
    - gemma2-9b-it (요청이 매우 간단하거나 반복적인 경우)
    - llama3-70b-8192 (일반적인 질문이나 가벼운 생성 작업)
    - deepseek-r1-distill-llama-70b-specdec (복잡하거나 창의적 사고가 필요한 고급 요청)


    - 사용자의 질문에는 답변하지 않습니다.
    - 먼저 왜 해당 모델을 선택했는지 간단히 설명하고, 아래 형식으로 **반드시 그대로 출력**해주세요:


    출력 예시:

    선택 이유: 사용자의 요청이 어떤 유형이며, 어떤 모델이 그에 적합한지 설명.
    선택한 모델 이름 : "gpt-4o-mini"

    '''

    select = llm_call(select_prompt)
    
    reason = select.split('\n')[0]
    
    if not select:
        raise "No model selected"
    
    model_name = re.search(r'\"(.*?)\"', select).group(1)
    
    print(f'Reasoning Select Model : {reason}')
    print(f'selected model : {model_name}')
    
    response = llm_call(user_input, model_name)
    
    return response

route(user_input3)

In [ ]:
print(f'Reasoning Select Model : {reason})
      

      
print(f'selected model : {model_name})

In [ ]:
select_prompt = f'''

user_input : {user_input1}


당신은 AI 라우팅 시스템입니다.
사용자의 요청을 보고 가장 적절한 AI 모델을 선택하세요.

선택 가능한 모델:
- gemma2-9b-it (요청이 매우 간단하거나 반복적인 경우)
- llama3-70b-8192 (일반적인 질문이나 가벼운 생성 작업)
- deepseek-r1-distill-llama-70b-specdec (복잡하거나 창의적 사고가 필요한 고급 요청)


- 사용자의 질문에는 답변하지 않습니다.
- 먼저 왜 해당 모델을 선택했는지 간단히 설명하고, 그 다음 아래 처럼 출력해주세요


출력 예시:

선택 이유: 사용자의 요청이 어떤 유형이며, 어떤 모델이 그에 적합한지 설명.
선택한 모델 이름 : "gpt-4o-mini"

'''

select = llm_call(select_prompt)

print(select)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from utils import llm_call

prompt = "다음 문장을 영어로 번역해주세요"
inputs = [
    "ChatGPT는 OpenAI가 개발한 대형 언어 모델입니다.",
    "AI 기술은 산업 전반에 혁신을 불러오고 있습니다.",
    "기후 변화는 글로벌 이슈 중 하나입니다."
]

def parallel(prompt: str, inputs: List[str], n_workers: int = 3) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(llm_call, f"{prompt}\nInput: {x}") for x in inputs]
        return [f.result() for f in futures]
    
    
impact_results = parallel(
    prompt,
    
    """
    ChatGPT는 OpenAI가 개발한 대형 언어 모델입니다.",
    "AI 기술은 산업 전반에 혁신을 불러오고 있습니다.",
    "기후 변화는 글로벌 이슈 중 하나입니다.
    
    """
    
)

for result in impact_results:
    print(result)

In [ ]:
from groq import AsyncGroq

In [ ]:
import re
from utils import llm_call_async, llm_call
import asyncio

async def run_llm_parallel(prompt_details):
    tasks = [llm_call_async(prompt['user_prompt'], prompt['model']) for prompt in prompt_details]
    responses = []
    
    for task in asyncio.as_completed(tasks):
        result = await task
        print("LLM 응답 완료:", result)
        responses.append(result)
    
    return responses



async def main():
    question = ("아래 문장을 자연스러운 한국어로 번역해줘:\n"
                "\"Do what you can, with what you have, where you are.\" — Theodore Roosevelt")
    
    parallel_prompt_details = [
        {"user_prompt": question, "model": "gemma2-9b-it"},
        {"user_prompt": question, "model": "llama3-70b-8192"},
        {"user_prompt": question, "model": "deepseek-r1-distill-llama-70b-specdec"},
    ]
    
    responses = await run_llm_parallel(parallel_prompt_details)
    
    aggregator_prompt = ("다음은 여러 개의 AI 모델이 사용자 질문에 대해 생성한 응답입니다.\n"
                         "당신의 역할은 이 응답들을 모두 종합하여 최종 답변을 제공하는 것입니다.\n"
                         "일부 응답이 부정확하거나 편향될 수 있으므로, 신뢰성과 정확성을 갖춘 응답을 생성하는 것이 중요합니다.\n\n"
                         "사용자 질문:\n"
                         f"{question}\n\n"
                         "모델 응답들:")
    
    for i in range(len(parallel_prompt_details)):
        aggregator_prompt += f"\n{i+1}. 모델 응답: {responses[i]}\n"
    
    print("---------------------------종합 프롬프트:-----------------------\n", aggregator_prompt)
    final_response = await llm_call_async(aggregator_prompt, model="llama3-70b-8192")
    print("---------------------------최종 종합 응답:-----------------------\n", final_response)

# 비동기 main 함수 실행
await main()

동기적으로 하나씩 순차적으로 출력

In [ ]:
from utils import llm_call

user_input = '''업무적 피로감으로 인해 편하게힐링 할 수 있는 곳으로 여행을 가고 싶어 
                여행지를 2가지만 추천해줘 추천 여행지, 여행 코스를 알려주고 그 이유도 알려줘
                나는 여러 관광지를 돌아다는것 보다는 경치좋고 특색있는 명소를 찾아다니는것을 좋아해
                아래 포맷대로 작성해줘 
                
                추천 여행지 :
                1. OO
                
                추천 코스:
                OO => OO => OO
                
                추천 이유:
                OO
                
                
                '''

prompt = [
    
    {'user' : user_input, 'model' : 'gemma2-9b-it'},
    {'user' : user_input, 'model' : 'llama3-70b-8192'},
    {'user' : user_input, 'model' : 'deepseek-r1-distill-llama-70b-specdec'}
    
]

for prom in prompt:
    
    print(f'{prom["model"]}')
    response = llm_call(prom['user'], prom['model'])
    
    print(response)

In [ ]:
asyncio.create_task

In [32]:
for i in range(len(prompt)):
    print(f'{prompt[i]["model"]}')


gemma2-9b-it
llama3-70b-8192
deepseek-r1-distill-llama-70b-specdec


In [ ]:
# 비동기 함수를 사용하여 여러 모델에 대한 요청을 병렬로 처리하는 예제입니다.
from utils import llm_call_async
import asyncio

user_input = '''
업무적 피로감으로 인해 편하게힐링 할 수 있는 곳으로 여행을 가고 싶어 
여행지를 2가지만 추천해줘 추천 여행지, 여행 코스를 알려주고 그 이유도 알려줘
나는 여러 관광지를 돌아다는것 보다는 경치좋고 특색있는 명소를 찾아다니는것을 좋아해
아래 포맷대로 작성해줘 

추천 여행지 :
1. OO

추천 코스:
OO => OO => OO

추천 이유:
OO
'''


async def parallel(prompt):
    tasks = []
    for prom in prompt:
        tasks.append(llm_call_async(prom['user'], prom['model']))
        
    responses = []
    for task in asyncio.as_completed(tasks):
        result = await task
        responses.append(result)
    
    return responses


async def main(user_input):
    
    prompt = [
    
    {'user' : user_input, 'model' : 'gemma2-9b-it'},
    {'user' : user_input, 'model' : 'llama3-70b-8192'},
    {'user' : user_input, 'model' : 'deepseek-r1-distill-llama-70b-specdec'}
    
    ]
    
    responses = await parallel(prompt)
    
    ensemble_prompt = f'''
입력된 텍스트은 사용자의 요청에 따라 출력한 여러가지 결과입니다. 
이 결과들을 종합하고 요약하여 사용자의 질문에 맞는 최종 결과를 출력해주세요.

사용자 질문:
    {user_input}

모델별 응답:
'''
                    
    for i in range(len(prompt)):
        ensemble_prompt += f'model :{prompt[i]["model"]}\n모델 응답: {responses[i]}\n'
        
    result = await llm_call_async(ensemble_prompt, model="gemma2-9b-it")
    
    print("=========================프롬프트==========================\n")
    print(ensemble_prompt)
    
    print("=========================출력 결과=========================")
    print(result)

# 비동기 main 함수 실행
await main(user_input)

KeyError: <coroutine object as_completed.<locals>._wait_for_one at 0x000002AF4C004A00>

In [ ]:
for i in range(len(prompt)):
    ensemble_prompt += f'model :{prompt[i]["model"]}\n모델 응답: {responses[i]}\n'

In [ ]:
''.join([f'model :{prompt[i]["model"]}\n모델 응답: {responses[i]}\n' for i in range(len(prompt))])